### Approximating the integral

It is easy to see that as we use large $n$ (i.e., more qubits) and thus larger $b_{\mbox{max}}$, we can approximate the integral better as illustrated below. 

In [1]:
import math

b_max = math.pi / 500  # upper limit of integral # 5
nbit = 10  # change this value to get discretized result closer to analytical results # 3

analyticResult = (b_max / 2.0 - math.sin(2 * b_max) / 4.0 ) / b_max  # the target integral can be analytically solved
print("Analytical Result:", analyticResult)

ndiv = 2 ** nbit  #number of discretization 
discretizedResult = 0.0
for i in range(ndiv):
    discretizedResult += math.sin(b_max / ndiv * (i + 0.5)) ** 2
discretizedResult = discretizedResult / ndiv
print("Discretized Result:", discretizedResult)

Analytical Result: 1.315936863217701e-05
Discretized Result: 1.3159365494765862e-05


In [2]:
import sys
sys.path.append('../../')

In [3]:
import sys
sys.path.append('../../')
#Preparing qiskit environment
from qiskit import QuantumRegister
from qiskit import execute
from qiskit import Aer
from scipy import optimize
import time
import numpy as np
import matplotlib.pyplot as plt
import pickle
seed = 42
np.random.seed(seed)

In [4]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import depolarizing_error
import qiskit.ignis.mitigation as mit
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter
from lib_grover import *

## Testing the algorithm with simulators

Let us run the algorithm with simulators

In [5]:
#setting the number of shots and Grover operators.
max_iter = 10
# number_grover_list = [0, 1, 2, 4, 8, 16, 32] # , 64, 128, 256]  # list of number of Grover operators
number_grover_list = [5 * i for i in range(10)]
shots_list = [8192] * len(number_grover_list) # list of number of shots
if len(shots_list) != len(number_grover_list):
    raise Exception(
        'The length of shots_list should be equal to the length of number_grover_list.'
    )

backend = Aer.get_backend('qasm_simulator')

In [6]:
noise_model = NoiseModel()
for qi in range(nbit+1):
    if qi >= 0:
        read_err = ReadoutError([[0.990, 0.010],[0.010,0.990]])
        noise_model.add_readout_error(read_err, [qi])

Finally, we can run the algorithm as below: 

In [7]:
qc_list = create_grover_circuit(number_grover_list, nbit,b_max)  # list of Grover circuits

In [8]:
t1 = time.time()
counts_lists = []
for i in range(max_iter): # set the seed as index i
    print("iter", i, "started")
    counts_list = run_grover(qc_list, number_grover_list, shots_list, backend, noise_model=noise_model, seed_transpiler=i, seed_simulator=i)  # list of number of grover operators
    counts_lists.append(counts_list)
    with open("pkls/hists_shots8192.pkl", "wb") as f:
        pickle.dump(counts_lists, f)
    print("iter", i, "finished")
    print()
t2 = time.time()
print(t2-t1)

iter 0 started
0 th round with 0 oracles
0.175645112991333 s
1 th round with 5 oracles
0.4711461067199707 s
2 th round with 10 oracles
0.8905930519104004 s
3 th round with 15 oracles
1.2431371212005615 s
4 th round with 20 oracles


KeyboardInterrupt: 

In [9]:
shots = 8192
qr = QuantumRegister(nbit+1)
mit_pattern = [[i] for i in range(nbit+1)]
meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')
meas_fitters = []
for i in range(max_iter):
    job = execute(meas_calibs, backend=Aer.get_backend('qasm_simulator'), shots=shots, noise_model=noise_model, seed_transpiler=i, seed_simulator=i)
    cal_results = job.result()
    meas_fitter = TensoredMeasFitter(cal_results, mit_pattern=mit_pattern)
    meas_fitters.append(meas_fitter)
    print("iter", i, "finished")

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
iter 10 finished
iter 11 finished
iter 12 finished
iter 13 finished
iter 14 finished
iter 15 finished
iter 16 finished
iter 17 finished
iter 18 finished
iter 19 finished
iter 20 finished
iter 21 finished
iter 22 finished
iter 23 finished
iter 24 finished
iter 25 finished
iter 26 finished
iter 27 finished
iter 28 finished
iter 29 finished
iter 30 finished
iter 31 finished
iter 32 finished
iter 33 finished
iter 34 finished
iter 35 finished
iter 36 finished
iter 37 finished
iter 38 finished
iter 39 finished
iter 40 finished
iter 41 finished
iter 42 finished
iter 43 finished
iter 44 finished
iter 45 finished
iter 46 finished
iter 47 finished
iter 48 finished
iter 49 finished
iter 50 finished
iter 51 finished
iter 52 finished
iter 53 finished
iter 54 finished
iter 55 finished
iter 56 finished
iter 57 finished
iter 58 finished
iter 59

In [10]:
with open("pkls/meas_fitters.pkl", "wb") as f:
    pickle.dump(meas_fitters, f)